In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
# os.chdir(r"C:\Users\Administrator.DAI-PC2\Desktop\CDAC\Practical Machine Learning\Kaggle\Flood Prediction\Dataset")

train = pd.read_csv("train.csv", index_col=0)
print(train.isnull().sum().sum())
test = pd.read_csv("test.csv")
print(test.isnull().sum().sum())

0
0


In [3]:
X_train = train.drop('FloodProbability', axis=1)
y_train = train['FloodProbability']
X_test = test.drop('id', axis=1)

In [4]:
# lr = LinearRegression()
# bagg = BaggingRegressor(lr,n_estimators=25, n_jobs=-1,random_state=24)

# bagg.fit(X_train,y_train)
# y_pred = bagg.predict(X_test)

el = ElasticNet()
bagg = BaggingRegressor(el,n_estimators=25, n_jobs=-1,random_state=24)

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

kfold = KFold(n_splits=5,shuffle=True, random_state=24)
print(bagg.get_params())

params = {'estimator__alpha': np.linspace(0.001,5,5),'estimator__l1_ratio': np.linspace(0,1,4)}
# n_jobs = The number of jobs to run in parallel for fit (Will use all CPU Cores; Multiprocessing)
gcv = GridSearchCV(bagg, param_grid=params, cv=kfold, n_jobs=-1)
gcv.fit(X_train,y_train)

best_model = gcv.best_estimator_
y_pred = best_model.predict(X_test)

{'bootstrap': True, 'bootstrap_features': False, 'estimator__alpha': 1.0, 'estimator__copy_X': True, 'estimator__fit_intercept': True, 'estimator__l1_ratio': 0.5, 'estimator__max_iter': 1000, 'estimator__positive': False, 'estimator__precompute': False, 'estimator__random_state': None, 'estimator__selection': 'cyclic', 'estimator__tol': 0.0001, 'estimator__warm_start': False, 'estimator': ElasticNet(), 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 25, 'n_jobs': -1, 'oob_score': False, 'random_state': 24, 'verbose': 0, 'warm_start': False}


In [ ]:
submit = pd.DataFrame({'id':test['id'],
                       'FloodProbability':y_pred})
submit.to_csv("sbt_bagging_regressor.csv", index=False)